## Generate HepMC file
I only store the tau lepton and the particles that creates it.

In [1]:
import csv
import pyhepmc
import pandas as pd
import numpy as np

In [3]:
# Load the event info
event_info = pd.read_csv("../data/genie_tau_decays_event_info.csv")
particle_info = pd.read_csv("../data/genie_tau_decays_particles.csv")

In [4]:
def csv2genevent(event: pd.DataFrame, nucleus_row=0, nutau_row=1, tau_row=2) -> pyhepmc.GenEvent:
    """This is a simplified version of the same function in plot_hepmc.
    In that file, all decay products are included in the decay chain (including the electron that is not in the csv file)
    Here, only the nucleus, neutrino and tau are included. This is because the tau will be decayed by Tauola."""
    evt = pyhepmc.GenEvent(pyhepmc.Units.GEV, pyhepmc.Units.MM)
    
    # Extract particles        
    nucleus = pyhepmc.GenParticle(pyhepmc.FourVector(*event.iloc[nucleus_row, 3:6], event.iloc[nucleus_row, 2]), event.iloc[nucleus_row, 1], 3)
    nutau = pyhepmc.GenParticle(pyhepmc.FourVector(*event.iloc[nutau_row, 3:6], event.iloc[nutau_row, 2]), event.iloc[nutau_row, 1], 3)
    # Set status to 1, since we want Tauola to decay it
    tau = pyhepmc.GenParticle(pyhepmc.FourVector(*event.iloc[tau_row, 3:6], event.iloc[tau_row, 2]), event.iloc[tau_row, 1], 1)

    nucleus_out4m = nucleus.momentum + nutau.momentum - tau.momentum
    # Set the outoging nucleus PDG.
    # NOTE this is not really accurate, since for deep inelastic scattering, the nucleus breaks up into many particles
    # If the incoming nucleus is a proton and the , the outgoing nucleus is a neutron
    # if nucleus.pdg == 2212:
    #     nucleus_out_pdg = 2112
    # # If incoming nucleus is an oxygen-16 nucleus, the outgoing nucleus is a nitrogen-16 nucleus
    # elif nucleus.pdg == 1000080160:
    #     nucleus_out_pdg = 1000090190
    # else:
    #     raise ValueError(f"Unknown nucleus PDG: {nucleus.pdg}")

    nucleus_out = pyhepmc.GenParticle(nucleus_out4m, 0, 1)
    # Create vertices
    interaction_vertex = pyhepmc.GenVertex()
    interaction_vertex.add_particle_in(nucleus)
    interaction_vertex.add_particle_in(nutau)
    interaction_vertex.add_particle_out(tau)
    interaction_vertex.add_particle_out(nucleus_out)
    evt.add_vertex(interaction_vertex)
    return evt

In [5]:
with pyhepmc.open("../data/tauola_input.dat", "w") as f:
    for event_num, event in particle_info.groupby("event_num"):
        evt = csv2genevent(event)
        evt.event_number = event_num
        f.write(evt)
        if event_num % 100 == 0:
            print(event_num)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [6]:
evt

<GenEvent momentum_unit=1, length_unit=0, event_number=76642, particles=4, vertices=1, run_info=None>

## GENIE events
These are GENIE events that I simulated on my own

In [19]:
genie_events = pd.read_csv("../data/genie_tau_pol_data.csv")
genie_events

,event_num,pdg,E,px,py,pz,polx,poly,polz
0,7,16,10.00000,0.000000,0.000000,10.00000,0.000000,0.000000,0.000000
1,7,1000080160,14.89510,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2,7,15,9.09355,-0.775883,-0.073440,8.88415,0.086999,0.008235,-0.996174
3,10,16,10.00000,0.000000,0.000000,10.00000,0.000000,0.000000,0.000000
4,10,1000080160,14.89510,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
50995,49994,1000080160,14.89510,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
50996,49994,15,5.04112,0.431838,0.070877,4.69725,-0.091538,-0.015024,-0.995688
50997,49997,16,10.00000,0.000000,0.000000,10.00000,0.000000,0.000000,0.000000
50998,49997,1000080160,14.89510,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000


In [20]:
assert (genie_events.groupby("event_num").count() == 3).all().all()

In [21]:
genie_events["event_num"].unique().size

17000

Convert events to HepMC format

In [22]:
with pyhepmc.open("../data/tauola_input_genie.dat", "w") as f:
    for event_num, event in genie_events.groupby("event_num"):
        evt = csv2genevent(event)
        evt.event_number = event_num
        f.write(evt)
        if event_num % 100 == 0:
            print(event_num)

100
200
300
400
500
800
1300
1500
1600
1900
3500
3600
3700
3900
4100
4400
4500
4700
5100
5500
5800
6100
6500
6900
7100
7600
7800
8000
8200
8600
9700
10100
10700
10900
11000
11200
11400
11600
11900
12200
12400
13100
13300
13900
14000
14100
14700
15000
15200
15500
15700
16100
16200
16300
16600
16800
18000
18100
18200
18500
18700
18800
20100
20200
20300
20500
20600
21500
21700
22000
22200
22600
23100
23400
23700
23800
25300
25700
26100
26300
26500
26900
27100
27500
27900
28000
28100
28400
28500
28600
28900
29200
29600
29800
29900
30200
30400
31000
31200
31300
31900
32200
32300
32400
32500
32800
32900
33000
33100
34500
35900
36500
36600
36800
37100
37500
37900
38200
38600
38800
39100
39300
39700
40400
40500
40600
40700
40900
41200
41300
41400
41600
41800
42000
42200
42900
43000
43300
43500
43600
43900
44100
44300
44500
44900
45000
45400
45800
46300
46400
47100
47400
47600
47800
47900
48100
48200
48300
48700
49100


Save polarization in a separate file

In [23]:
# Calculate the polarization vector norm
polarization = genie_events.groupby("event_num", as_index=False).last()
# Perform normalization twice to fix numerical errors where the polarization is >1
for i in range(2):
    polarization["polarization"] = np.sqrt(polarization["polx"]**2 + polarization["poly"]**2 + polarization["polz"]**2)
    print((polarization["polarization"] > 1).sum())
    
    polarization.loc[polarization["polarization"] > 1, "polx":"polz"] = (
        polarization.loc[polarization["polarization"] > 1, "polx":"polz"].values.T 
        / polarization.loc[polarization["polarization"] > 1, "polarization"].values
    ).T

assert (np.sqrt(polarization["polx"]**2 + polarization["poly"]**2 + polarization["polz"]**2) > 1).sum() == 0

8506
135


In [24]:
polarization.drop("polarization", axis=1).to_csv("../data/genie_pol.csv", index=False)